In [ ]:
import requests

EnjinEmail = "ENTER EMAIL HERE"
EnjinPassword = "ENTER PASSWORD HERE"

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "cache-control": "no-cache"
          }


def run_query(query): # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post('https://cloud.enjin.io/graphql', json={"query": query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request, query))

        
# The GraphQL query (with a few aditional bits included) itself defined as a multi-line string.       
loginQuery = """
query login{
  EnjinOauth (
    email: "%s",
    password: "%s"
  ) {
    id,
    name,
    email,
    access_tokens
  }
}
""" % (EnjinEmail, EnjinPassword)

result = run_query(loginQuery) # Execute the query
result

In [ ]:
# Add Auth token to your headers
headers.update({"Authorization": "Bearer " + result['data']['EnjinOauth']['access_tokens'][0]['access_token']})
headers


In [ ]:
# Get X-App-Id
appName = "Makerverse" #ENTER APP NAME as a string
    
appQuery = """
query searchAppId{
  EnjinApps (
    name: ""
    )
    {
    id
    name
    }
  }
""" # % (appName)
AppIDResult = run_query(appQuery) # Execute the query
for item in AppIDResult['data']['EnjinApps']:
    if item['name'] == appName:
        AppID = item['id']
AppID

In [ ]:
# Add X-App-Id to your headers
headers.update({"X-App-ID": str(AppID)})
headers

In [ ]:
#view identities
IDQuery = """
query viewIdentities{
  EnjinIdentities (
    pagination: {
      page: 1,
      limit: 50
    }
  ) {
    id
    app {
      name
    }
    linking_code
    enj_allowance
    ethereum_address
  }
}
"""

ID = run_query(IDQuery) # Execute the query
ID

In [ ]:
identity_id = ID['data']['EnjinIdentities'][0]['id']

In [ ]:
# create token example
# set variables below
tokenName = "ENTER TOKEN NAME HERE"
meltFeeRatio = 25 # ENTER VALUE HERE set your melt fee ratio in %
meltValue = 1 # ENTER VALUE HERE set melt value in ENJ
transferFee = 1 # ENTER VALUE HERE set your transfer fee in ENJ
initialReserve = 1 # ENTER VALUE HERE set initial reserve
totalSupply = 1 # ENTER VALUE HERE set total supply
nonFungible = "false" #"true"/"false"


In [ ]:
# DO NOT CHANGE
# parse variables for injection into query

identity_id = str(identity_id) # parse to string fror query
meltFeeRatio = meltFeeRatio * 100 # DO NOT EDIT translate melt fee ratio to value necessary for query
meltFeeRatio = str(meltFeeRatio) # DO NOT EDIT parse to string for query
meltValue = meltValue * 10**18 # DO NOT EDIT translate melt value to value necessary for query
meltValue = str(meltValue)
transferFee = transferFee * 10**18 # DO NOT EDIT translate transfer fee to value necessary for query
transferFee = str(transferFee)
initialReserve = str(initialReserve) # DO NOT EDIT translate initial reserve to value necessary for query
totalSupply = str(totalSupply) # DO NOT EDIT translate total supply to value necessary for query

In [ ]:
# run create Token query 
createTokenQuery = """
mutation createTokenRequest{
  CreateEnjinRequest (
    identity_id: %s,
    type: CREATE,
    create_token_data: {
      name: "%s", 
      totalSupply: %s,
      initialReserve: %s,
      supplyModel: FIXED,
      meltValue: "%s",
      meltFeeRatio: %s,
      transferable: PERMANENT,
      transferFeeSettings: {
        type: PER_TRANSFER,
        token_id: "0",
        value: "%s",
      }
      nonFungible: %s,
    }
  ) {
    id,
    encoded_data
  }
}
""" % (identity_id, tokenName, totalSupply, initialReserve, meltValue, meltFeeRatio, transferFee, nonFungible)


result = run_query(createTokenQuery) # Execute the query
result

In [ ]:
# after approving the token creation 
# get the token ID for minting
viewTokenQuery = """
query viewTokens{
  EnjinTokens (
    name: "%s",
    pagination: {
      page: 1,
      limit: 50
    }
  ) {
    token_id
    name
    creator
    meltValue
    meltFeeRatio
    meltFeeMaxRatio
    supplyModel
    totalSupply
    circulatingSupply
    reserve
    transferable
    nonFungible
    blockHeight
    markedForDelete
    created_at
    updated_at
    availableToMint
    itemURI
  }
}
""" % (tokenName)

tokenIDResult = run_query(viewTokenQuery) # Execute the query
tokenID = tokenIDResult['data']['EnjinTokens'][0]['token_id']
tokenID


In [ ]:
# add token metadata
metadataURI = "https://raw.githubusercontent.com/PatrickMockridge/MakerverseTokenMetadata/master/Cubespawn%20Patrons%20Token.json" #enter metadata URI as a string
setMetadataQuery = """
mutation setItemURI{
  CreateEnjinRequest (
    identity_id: %s,
    type: SET_ITEM_URI,
    set_item_uri_data: {
      token_id: "%s",
      token_index: 0,
      item_uri: "%s"
    }
  ) {
    id,
    encoded_data
  }
}
""" % (identity_id tokenID, metadataURI)

setMetadataURI = run_query(setMetadataQuery) # Execute the query
setMetadataURI

In [ ]:
address_list = # IMPORT FROM DATA MUNGE
value_list = # IMPORT FROM DATA MUNGE
mintTokenQuery = """
mutation mintFungibleItems {
  CreateEnjinRequest (
    identity_id: %s,
    type: MINT,
    mint_token_data: {
      token_id: "%s",
      recipient_address_array: [
        "%s"
      ]
      value_array: [
        %s
      ]
    }
  ) {
    id,
    encoded_data
  }
}
""" % (identity_id, tokenID, address_list, value_list)

mintResult = run_query(mintTokenQuery) # Execute the query
mintResult
